In [1]:
import tweepy
from textblob import TextBlob
import yaml

In [2]:
with open("../config.yml", 'r') as ymlfile:
    twitter_config = yaml.load(ymlfile)

/Users/deniel.horvatic/repository/machine-learning-masterclass/project_12_sentiment_analysis/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [3]:
# Step 1 - Authenticate
consumer_key = twitter_config['twitter']['consumer_key']
consumer_secret = twitter_config['twitter']['consumer_secret']

access_token= twitter_config['twitter']['access_token']
access_token_secret= twitter_config['twitter']['access_token_secret']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [4]:
api = tweepy.API(auth)

In [5]:
#Step 3 - Retrieve Tweets
public_tweets = api.search('Bad')

#CHALLENGE - Instead of printing out each tweet, save each Tweet to a CSV file
#and label each one as either 'positive' or 'negative', depending on the sentiment 
#You can decide the sentiment polarity threshold yourself


for tweet in public_tweets:
    print(tweet.text)
    print('Analysis')
    #Step 4 Perform Sentiment Analysis on Tweets
    analysis = TextBlob(tweet.text)
    if analysis.sentiment.polarity >= 0.2:
        print('Positiv')
    elif analysis.sentiment.polarity <= -0.2:
        print('Negativ')
    else:
        print('Neutral')
    print("")

@lutosinGBELA I’m actually crying at how bad it is. Omg even I can do better 😭😭
Analysis
Neutral

@GeriHalliwell I know how bad must it feel wanting to quit for the second time and mostly for the same reason and n… https://t.co/iAqW3uVk7W
Analysis
Neutral

@nathan_quao i hav to disagree wt u, smalling had a pretty decent game. i can only assume u r pissed at de own goal… https://t.co/fV72q4X6p1
Analysis
Neutral

Episodios do caso Evandro que deixam a gente na bad pra caralho
Analysis
Negativ

RT @EnRouteAnfield: Aguero - 19 goals 
"Best form of his life" 

Salah - 17 goals 
"Out of form, bad season" 

Sterling - 15 goals 
"Best f…
Analysis
Positiv

A bad bitch will get up early, do her self care routine, put herself together well, + handle her business, then sti… https://t.co/CMgeWeh5oS
Analysis
Negativ

@g_jersey2983 @Public_Citizen @AOC The greatest case against Christianity right now is the acts of American Christi… https://t.co/1jLSq67gpe
Analysis
Positiv

RT @c0zytapes: I wanna be

# Playfield for Twitter Client

In [6]:
'''
Generic Twitter Client
'''
class TwitterClient:
    def __init__(self):
        consumer_key = twitter_config['twitter']['consumer_key']
        consumer_secret = twitter_config['twitter']['consumer_secret']

        access_token= twitter_config['twitter']['access_token']
        access_token_secret= twitter_config['twitter']['access_token_secret']
        
        self.auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        self.auth.set_access_token(access_token, access_token_secret)
        self.api = tweepy.API(self.auth)
        self.tweet_n_max = 100
        
    def get_tweets(self, query):
        tweets = []
        raw_tweets = self.api.search(q=query, count=self.tweet_n_max)

        for tweet in raw_tweets:
            parsed_tweet = {}
            parsed_tweet['text'] = tweet.text
            parsed_tweet['user'] = tweet.user.screen_name
            sentiment = self.get_sentiment(tweet.text)
            parsed_tweet['sentiment_status'] = sentiment[0]
            parsed_tweet['sentiment_polarity'] = sentiment[1]
            
            tweets.append(parsed_tweet)
        
        return tweets
    
    def get_sentiment(self, tweet_text):
        analysis = TextBlob(tweet_text)
        if analysis.sentiment.polarity > 0:
            return ('positive',analysis.sentiment.polarity)
        elif analysis.sentiment.polarity < 0:
            return ('negative',analysis.sentiment.polarity)
        return ('neutral',analysis.sentiment.polarity)

client = TwitterClient()
client


In [7]:
client.get_tweets('happy')

[{'text': 'RT @kittenhae: happy birthday hyukjae you absolute angel!! 💕 #HappyEunhyukDay #사랑에_사랑을_더한_혁재의_봄 https://t.co/BvEwE2OdHm',
  'user': 'eunhaebaka',
  'sentiment_status': 'positive',
  'sentiment_polarity': 0.55625},
 {'text': 'RT @jokermovie: Put on a happy face. #JokerMovie - in theaters October 4. https://t.co/TxF3Jqxjjr',
  'user': 'haydenblu7_',
  'sentiment_status': 'positive',
  'sentiment_polarity': 0.8},
 {'text': 'RT @MybabyH: @SJofficial @AllRiseSilver #HappyEunhyukDay \n#SUPERJUNIOR @SJofficial \n#EUNHYUK\nHappy birthday Lee Hyukjae oppa🎂🎁🎉 wish you al…',
  'user': 'toolai4',
  'sentiment_status': 'positive',
  'sentiment_polarity': 0.8},
 {'text': 'I’m an soo happy once a week going to my internship now 😍💕 ONLY  Wednesday’s now',
  'user': 'PrettyKieana_xo',
  'sentiment_status': 'positive',
  'sentiment_polarity': 0.4},
 {'text': "An hour ago you were both happy and now you're setting yourselves free from each other.",
  'user': 'Queentified',
  'sentiment_status'

In [8]:
client.get_sentiment('good,good,good,')

('neutral', 0.0)

In [9]:
from flask import Flask, request, render_template, jsonify 
from werkzeug.wrappers import Request, Response

app = Flask(__name__)
api = TwitterClient()  
@app.route('/')
def index():
    return render_template('index.html')


@app.route('/tweets')
def tweets():
        query = request.args.get('query')
        tweets = api.get_tweets(query)
        return jsonify({'data': tweets, 'count': len(tweets)})


port = 5005
#app.run(host="0.0.0.0", port=port, debug=True)
from werkzeug.serving import run_simple
run_simple('0.0.0.0', port, app)

 * Running on http://0.0.0.0:5005/ (Press CTRL+C to quit)
